In [1]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
model_name = "dbmdz/bert-base-french-europeana-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/227k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

In [3]:
lr = 2e-5
epochs =  6
batch_size = 5
max_seq_len = 75

test_frac = 0.1

In [4]:
import os

def set_seed(seed=106052):
    """Set seed for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [5]:
class CEFRDataset(Dataset):
    """Classification dataset, built on top of pytorch dataset object
    """
    
    def __init__(self, texts, labels):
        
        self.encoder = LabelEncoder()
        print(self.encoder.__dict__)
        self.texts = texts
        self.labels = self.encoder.fit_transform(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=max_seq_len,
            truncation=True,
            return_tensors="pt",
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()
        label = torch.tensor(label)

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
            "labels": label,
        }

    def get_labels(self):
        return self.labels

In [6]:
def train(train_set, valid_set, epochs=10, warmup_size=0.1, lr=1e-3, batch_size=16):
    model = get_model(model_name)
    optim = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler(
        optim, warmup_size, round(len(train_set) / batch_size * epochs)
    )
    training_args = get_training_args(epochs, batch_size)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set,
        optimizers=[optim, scheduler],
        compute_metrics=compute_accuracy,
    )
    trainer.train()
    trainer.save_model()
    return trainer

In [7]:
def get_model(pretrained_checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_checkpoint, num_labels=6
    )
    return model.to(device)

In [8]:
os.environ["WANDB_DISABLED"] = "true"


def get_scheduler(optimizer, warmup_size, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=round(total_steps * warmup_size),
        num_training_steps=total_steps,
    )
    return scheduler


def get_training_args(epochs, batch_size):
    return TrainingArguments(
        output_dir="./b",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_steps=50,
        fp16=False,
        evaluation_strategy="epoch",
        eval_accumulation_steps=1,
        report_to=None,
        save_total_limit=1,
        load_best_model_at_end=True
    )


def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}



In [9]:
lr = 2e-5
epochs =  7
batch_size = 8
max_seq_len = 512

In [10]:
def split_valid(df, frac=0.01):
    
    val = pd.DataFrame()
    val["text"] = ""
    val["label"] = -1
    
    for i in df.label.unique():
        val = pd.concat([val, df[df.label == i].sample(frac=frac)])
        
    return df[~df.index.isin(val.index)].reset_index(drop=True) , val.reset_index(drop=True)

In [11]:
train_set_df = pd.read_csv("../input/fr-cerfr/dataset_to_be_annotated_-_dataset_to_be_annotated.csv")
train_set_df.drop("Unnamed: 3", axis=1, inplace=True)
train_set_df = train_set_df[pd.notnull(train_set_df['cefr'])]
train_set_df = train_set_df.reset_index(drop=True)
train_set_df.columns=["text","lens","label"]
train_set_df = train_set_df[train_set_df.label != "-"]
train_set_df = train_set_df[["text", "label"]]

In [12]:
A1_A2=pd.read_csv("../input/collecteda1a2/frr.csv - Feuil1.csv")
A1_A2=A1_A2.dropna()
A1_A2.columns=["text","label"]
A1_A2 = A1_A2.reset_index(drop=True)
A1A2=pd.read_csv("../input/collecteda1a2/complete.csv.csv")
A1__A2=pd.read_csv("../input/a1-a2-ex/A1_A2_tran.csv")

In [13]:
A1_A2["label"].value_counts()

B1                        43
A2                        24
B2                        14
A1                        13
 A2                        1
texte language familer     1
B1 (répétition 41)         1
B25                        1
Name: label, dtype: int64

In [14]:
"""separating the new dataset into sentences with keeping the same labels"""
df=pd.DataFrame({"text":[] , "label":[]})
L=[]
l=[]
for i in range(A1__A2.shape[0]):
    P=[]
    nb=0
    K=[]
    label=A1__A2.loc[i,"label"]
    P.append(A1__A2.loc[i,"text"].split("."))
    P=[i for p in P for i in p]
    for i in P:
        L.append(i)
    nb=len(P)
    for j in range(nb):
        l.append(label)

    

df["text"]=L
df["label"]=l

In [15]:
df.label.value_counts()

A2    5811
A1    4236
Name: label, dtype: int64

In [16]:
"""new dataframe that contain only A1 , A2 , B1 , B2"""
txt=[]
label=[]
nb=0
nbb=0
for i in range(A1_A2.shape[0]):
    if A1_A2.loc[i,"label"]=="A2":
        txt.append(A1_A2.loc[i,"text"])
        label.append(A1_A2.loc[i,"label"])
       
    if  A1_A2.loc[i,"label"]=="A1": 
            txt.append(A1_A2.loc[i,"text"])
            label.append(A1_A2.loc[i,"label"])
#     elif  A1_A2.loc[i,"label"]=="B1": 
#             txt.append(A1_A2.loc[i,"text"])
#             label.append(A1_A2.loc[i,"label"])
#     elif  A1_A2.loc[i,"label"]=="B2": 
#             txt.append(A1_A2.loc[i,"text"])
#             label.append(A1_A2.loc[i,"label"])
A1_A2_=pd.DataFrame({"text":txt,"label":label})


In [17]:
A1_A2_.label.value_counts()

A2    24
A1    13
Name: label, dtype: int64

In [18]:
# """new dataframe that contain only A1 , A2 , B1 , B2"""
# txt=[]
# label=[]
# nb=0
# nbb=0
# for i in range(A1_A2.shape[0]):
# #     if A1_A2.loc[i,"label"]=="A2":
# #         txt.append(A1_A2.loc[i,"text"])
# #         label.append(A1_A2.loc[i,"label"])
       
#     if  A1_A2.loc[i,"label"]=="A1": 
#             txt.append(A1_A2.loc[i,"text"])
#             label.append(A1_A2.loc[i,"label"])
# #     elif  A1_A2.loc[i,"label"]=="B1": 
# #             txt.append(A1_A2.loc[i,"text"])
# #             label.append(A1_A2.loc[i,"label"])
# #     elif  A1_A2.loc[i,"label"]=="B2": 
# #             txt.append(A1_A2.loc[i,"text"])
# #             label.append(A1_A2.loc[i,"label"])
# A1_A2_=pd.DataFrame({"text":txt,"label":label})

In [19]:
# """try to translate All A2 to A1 , because the model coundt catch A1 ,""" "failed"
# for i in range(A1__A2.shape[0]):
#     if A1__A2.loc[i,"label"]=="A2":
#         A1__A2.loc[i,"label"]="A1"

In [20]:
A1_A2_["label"].value_counts()

A2    24
A1    13
Name: label, dtype: int64

In [21]:
df["label"].value_counts()

A2    5811
A1    4236
Name: label, dtype: int64

In [22]:
train_set_df=pd.concat([train_set_df , A1_A2_], ignore_index=True)

In [23]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_set_df.text = train_set_df.text.apply(lambda x: x.replace("\r", "").replace("\n", " "))

# extra_df = pd.read_csv("../input/frenchcefr/french_mike_june.csv")
# extra_df.columns = ["text", "label", "label_"]
# extra_df = extra_df[["text", "label"]]
# extra_df.text = extra_df.text.astype(str)
#train_set_df = pd.concat([train_set_df, extra_df]).reset_index(drop=True)

train_set_df, valid_set_df = split_valid(train_set_df)

In [24]:
for i in range(train_set_df.shape[0]):
    if train_set_df.loc[i,"label"]=="c1":
        train_set_df.loc[i,"label"]="C1"

In [25]:
train_set_df.label.unique()

array(['C1', 'B2', 'B1', 'A2', 'C2', 'A1'], dtype=object)

In [26]:
train_set_df.label.value_counts()

B2    91
B1    50
C1    37
A2    28
A1    13
C2    12
Name: label, dtype: int64

In [27]:
train_set_df.label = le.fit_transform(train_set_df.label)
valid_set_df.label = le.transform(valid_set_df.label)

In [28]:
train_set_df.label.unique()

array([4, 3, 2, 1, 5, 0])

In [29]:
valid_set_df.label.nunique()

1

In [30]:
# train_set_df = train_set_df.sample(frac=1)

In [31]:
from tqdm import tqdm 

def predict(model, text):
    
    preds = []
    
    for i in tqdm(range(len(text))):
        tokenized = tokenizer(text[i:i+1], return_tensors="pt", truncation=True, max_length=512).to("cuda")
        pred = model(**tokenized)
        preds.append(pred.logits.argmax(-1).item())

    return preds

In [32]:
train_set = CEFRDataset(train_set_df["text"], train_set_df["label"])
valid_set = CEFRDataset(valid_set_df["text"], valid_set_df["label"])


trainer_second = train(train_set, valid_set, epochs=epochs, warmup_size=0.2, lr=lr, batch_size=batch_size)
model = trainer_second.model

{}
{}


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificat

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,No log,2.160720,0.000000,0.039800,25.107000
2,1.630900,3.411439,0.000000,0.030900,32.382000
3,1.630900,3.781452,0.000000,0.052400,19.068000
4,1.100500,4.148752,0.000000,0.031600,31.638000
5,1.100500,4.421740,0.000000,0.035200,28.430000
6,0.789200,4.594963,0.000000,0.032000,31.267000
7,0.602800,4.577701,0.000000,0.032400,30.910000


In [33]:
valid_set_df["preds"] = train_set.encoder.inverse_transform(predict(model, valid_set_df.text.tolist()))
valid_set_df.columns = ["text", "cefr", "preds",] 

100%|██████████| 1/1 [00:00<00:00, 51.13it/s]


In [34]:
lingua = pd.read_csv("../input/lingua/fr_lingua.csv")
lingua["preds"] = train_set.encoder.inverse_transform(predict(model, lingua.text.tolist()))

100%|██████████| 111/111 [00:01<00:00, 80.65it/s]


In [35]:
lingua.shape

(111, 3)

In [36]:
lingua.cefr.value_counts()

B1    60
A2    26
A1    13
B2    12
Name: cefr, dtype: int64

In [37]:
lingua.preds.unique()

array([1, 3])

In [38]:
def compute_average_distance(df, col_name="cefr") :
    
    labels = ["A1", "A2", "B1", "B2", "C1", "C2"]
    return (df[col_name].apply(lambda x: labels.index(x)) - df.preds.apply(lambda x: labels.index(x))).abs().mean()


In [39]:
lingua.preds.value_counts()

1    68
3    43
Name: preds, dtype: int64

In [40]:
lingua["preds"] = le.inverse_transform(lingua.preds)

In [41]:
lingua.cefr.value_counts()

B1    60
A2    26
A1    13
B2    12
Name: cefr, dtype: int64

In [42]:
lingua.preds.value_counts()

A2    68
B2    43
Name: preds, dtype: int64

In [43]:
print("Distance: ")
print(compute_average_distance(lingua, "cefr"))

print(f"Lingua accuracy: {(lingua['preds'] == lingua['cefr']).mean()}")

Distance: 
0.8738738738738738
Lingua accuracy: 0.2702702702702703


In [44]:
# lingua.columns=["text","cefr","preds:A1_A2_from_trans_data&new_data,acc:0.5"]
# lingua.to_csv("lingua1.csv", index=False)

****

****